## Comparing models for our different tasks

このノートブックでは、別のモデルである Flan-T5-large を Granite-7B-Instruct と並行して使用し、その動作を確認します。

Flan-T5-Large は確かに小さく、GPU なしで実行され、4 GB の RAM しか使用しませんが、タスクに十分対応できるでしょうか?

### 要件とインポート

ラボの指示に従って起動する適切なワークベンチ イメージを選択した場合は、必要なライブラリがすべてすでに用意されているはずです。そうでない場合は、次のセルの最初の行のコメントを解除して、適切なパッケージをすべてインストールします。

In [ ]:
# 適切なワークベンチ イメージを選択していない場合、またはこのノートブックをワークショップ環境外で使用している場合にのみ、次の行のコメントを解除します。
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt

import json
import time

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceTextGenInference, VLLMOpenAI

### Langchain パイプライン

ここでも、Langchain を使用して要約パイプラインを定義します。

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://granite-7b-instruct-predictor.ic-shared-llm.svc.cluster.local:8080"

# LLM definition
llm = VLLMOpenAI(           # 私たちは vLLM OpenAI 互換 API クライアントを使用しています。ただし、モデルは OpenAI ではなく OpenShift AI 上で実行されています。
    openai_api_key="EMPTY",   # そのため、これには OpenAI キーは必要ありません。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="granite-7b-instruct",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Flan-T5-Large LLM Inference Server URL
inference_server_url_flan_t5 = "http://llm-flant5.ic-shared-llm.svc.cluster.local:3000/"

# LLM definition
llm_flant5 = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url_flan_t5,
    max_new_tokens=96,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.01,
    repetition_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

 **template** は両方のモデルで同じになります。

In [ ]:
template="""<|system|>
You are a helpful, respectful and honest assistant.
Always assist with care, respect, and truth. Respond with utmost utility yet securely.
Avoid harmful, unethical, prejudiced, or negative content. Ensure replies promote fairness and positivity.
I will give you a text, then ask a question about it. Give a precise and as concise as possible answer to this question.
<|user|>
### TEXT:
{text}

### QUESTION:
{query}

### ANSWER:
<|assistant|>
"""
prompt = PromptTemplate(input_variables=["input"], template=template)

そして、2 つのモデルをクエリするために使用する 2 つの **会話** オブジェクトを作成できるようになりました。

In [ ]:
conversation = prompt | llm
conversation_flant5 = prompt | llm_flant5

これで、モデルをクエリする準備ができました。

この例では、1 つのクレームのみをクエリして、何が起こるかを確認します。 もちろん、別のクレームでも自由に試してください。

In [ ]:
filename = 'claims/claim1.json'

# JSONファイルを開く
claims = {}
with open(filename, 'r') as file:
    data = json.load(file)
claims[filename] = data

# クレーム内容とクエリのリスト
text_input = f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}"
sentiment_query = "What is the sentiment of the person sending this claim?"
location_query = "Where does the event the claim is related to happen?"
time_query = "When does the event the claim is related to happen?"

# クレームの解析
print(f"***************************")
print(f"* Claim: {filename}")
print(f"***************************")
print("Original content:")
print("-----------------")
print(f"Subject: {claims[filename]['subject']}\nContent:\n{claims[filename]['content']}\n\n")
print('Analysis with Granite-7B-Instruct:')
print("--------")
start_granite = time.time()
print(f"- Sentiment: ")
conversation.invoke(input={"text": text_input, "query": sentiment_query});
print("\n- Location: ")
conversation.invoke(input={"text": text_input, "query": location_query});
print("\n- Time: ")
conversation.invoke(input={"text": text_input, "query": time_query});
print("\n\n                          ----====----\n")
end_granite = time.time()
print('Analysis with Flan-T5-Large:')
print("--------")
start_flan = time.time()
print(f"- Sentiment: ")
conversation_flant5.invoke(input={"text": text_input, "query": sentiment_query});
print("\n- Location: ")
conversation_flant5.invoke(input={"text": text_input, "query": location_query});
print("\n- Time: ")
conversation_flant5.invoke(input={"text": text_input, "query": time_query});
print("\n\n                          ----====----\n")
end_flan = time.time()

print(f"Granite analysis time: {end_granite - start_granite:.2f} seconds")
print(f"Flan analysis time: {end_flan - start_flan:.2f} seconds")

ご覧のとおり、Flan-T5-Large は 7 億 7000 万のパラメータ モデルのみであるため、一部の結果を生成するのが速い可能性があります。ただし、それらの結果は精度や詳細度が低くなります。したがって、ある程度は機能しますが、結果は 70 億のパラメータである Granite-7B-Instruct の結果には遠く及びません。

LLM を使用する際の秘訣は、必要なパフォーマンスと精度、およびそれに伴うコストとともに必要なリソースとの間で適切なバランスを見つけることです。

したがって、データが変更されたりモデルが進化したりしても、常に期待どおりの動作が得られるように、信頼性チェックを実施することが重要です。